Part 1.2

Below is the functions we implemented to use in Part 2.2

In [ ]:
import random

LAMBDA = 1.0                # arrival rate (patients/hour)           (value for all groups)
MU_T = 0.476190476          # triage nurse service rate              (value for group 3)
MU_S = 0.16                 # stable home-care rate                  (value for all groups)
MU_CB = 0.118518519         # hospital bed service rate              (value for group 3)
P_STABLE = 0.2              # Probability that a patient is stable   (value for group 3)
P_CRITICAL = 1 - P_STABLE   # Probability that a patient is critical (value for group 3)

S = 3  # number of triage nurses        (value for group 3)
K = 9  # number of hospital beds        (value for group 3)

MY_SEED = 4040800189        # random seed for group 3 (2020400108 + 2020400081)
random.seed(MY_SEED)

def GenerateInterarrival():             # returns exponentially distributed interarrival time
    return random.expovariate(LAMBDA)

def GenerateNurseServiceTime():         # returns exponentially distributed service time for triage nurse   
    return random.expovariate(MU_T)

def GenerateHospitalHealingTime():      # returns exponentially distributed healing time for a patient who is hospitalized
    return random.expovariate(MU_CB)

def GenerateHomeHealingTime(condition='s'):   # returns exponentially distributed healing time for a patient who is home

    if condition == 's':                # If stable patient at home, use MU_S
        return random.expovariate(MU_S)
    else:                               # If critical patient forced to go home, use MU_CB / alpha, which will take longer
        alpha = random.uniform(1.25, 1.75)
        return random.expovariate(MU_CB / alpha)

FEL = []            # Future Event List (FEL) - list of events to be processed

                            # State Variables
currentTime = 0.0           # current simulation time
numberInTriageQueue = 0     # number of patients in triage queue
busyNurses = 0              # number of busy triage nurses
occupiedBeds = 0            # number of occupied hospital beds

                            # Additional Counters/Statistics:
totalPatientsArrived = 0    # total number of patients arrived
totalPatientsHealed = 0     # total number of patients healed
totalPatientsTreated = 0    # total number of patients treated in hospital
rejectedCriticalCount = 0   # number of critical patients sent home

def Arrival(event):
    global currentTime, numberInTriageQueue, busyNurses, totalPatientsArrived  

    currentTime = event[0]      # 1) Update simulation clock to this event's time

    totalPatientsArrived += 1   # 2) Increment total patients arrived

    interarrival_time = GenerateInterarrival()      # 3) Schedule the next arrival

    next_arrival_time = currentTime + interarrival_time     # Calculate next arrival time
    FEL.append((next_arrival_time, 'Arrival', None, {}))    # Add next arrival to FEL

    if busyNurses < S:          # 4) Check if a nurse is available
        busyNurses += 1         # If a nurse is available, increment busy nurses
        service_time = GenerateNurseServiceTime()           # Generate service time for the patient
        departure_triage_time = currentTime + service_time  # Calculate departure time

        FEL.append((departure_triage_time, 'DepartureTriage', None, {}))    # Add to FEL
    else:                       # 4) If no nurses are available, patient must wait
        numberInTriageQueue += 1

def DepartureTriage(event):
    global currentTime, numberInTriageQueue, busyNurses, occupiedBeds, rejectedCriticalCount

    currentTime = event[0]          # 1) Update clock to this event's time

    if numberInTriageQueue > 0:     # 2) If there are patients waiting in triage queue
        numberInTriageQueue -= 1    # Decrement the queue count
        service_time = GenerateNurseServiceTime()       # Generate service time for the next patient
        departure_time = currentTime + service_time     # Calculate departure time

        FEL.append((departure_time, 'DepartureTriage', None, {}))   # Add to FEL
    else:                           # 2) If no patients are waiting, decrement busy nurses    
        busyNurses -= 1

    if random.random() < P_STABLE:  # 3) If the patient is stable, recover at home
        home_time = GenerateHomeHealingTime(condition='s')      # Generate home healing time 

        FEL.append((currentTime + home_time, 'RecoveryHome', None, {'isStable': True})) # Add to FEL
    else:                           # 3) If the patient is critical 
        if occupiedBeds < K:        # If a hospital bed is available, admit the patient
            occupiedBeds += 1
            hospital_healing_time = GenerateHospitalHealingTime()
            FEL.append((currentTime + hospital_healing_time, 'TreatedAtHospital', None, {}))
        else:                       # If no bed is available, send the patient home
            rejectedCriticalCount += 1
            home_time = GenerateHomeHealingTime(condition='c')
            FEL.append((currentTime + home_time, 'RecoveryHome', None, {'isStable': False}))

def TreatedAtHospital(event):
    global currentTime, occupiedBeds

    currentTime = event[0]      # 1) Update clock to this event's time
    occupiedBeds -= 1           # 2) Decrement the number of occupied beds


def RecoveryHome(event):
    global currentTime

    currentTime = event[0]      # 1) Update clock to this event's time


def get_next_event():
    FEL.sort(key=lambda e: e[0])    # Sort the FEL by event time
    return FEL.pop(0)               # remove and return the first event


: 

Part 2.2

Initial conditions can be adjusted in the code according to the preferences.

In [ ]:
import random
import math
import numpy as np
import scipy.stats as stats

# Global Variables
LAMBDA = 1.0                # arrival rate (patients/hour)           (value for all groups)
MU_T = 0.476190476          # triage nurse service rate              (value for group 3)
MU_S = 0.16                 # stable home-care rate                  (value for all groups)
MU_CB = 0.118518519         # hospital bed service rate              (value for group 3)
P_STABLE = 0.2              # Probability that a patient is stable   (value for group 3)
P_CRITICAL = 1 - P_STABLE   # Probability that a patient is critical (value for group 3)

S = 3  # number of triage nurses        (value for group 3)
K = 9  # number of hospital beds        (value for group 3)

SEED = 4040800189           # random seed for group 3 (2020400108 + 2020400081)
random.seed(SEED)           # set the random seed


def GenerateInterarrival():             # returns exponentially distributed interarrival time
    return random.expovariate(LAMBDA)

def GenerateNurseServiceTime():         # returns exponentially distributed service time for triage nurse   
    return random.expovariate(MU_T)

def GenerateHospitalHealingTime():      # returns exponentially distributed healing time for a patient who is hospitalized
    return random.expovariate(MU_CB)

def GenerateHomeHealingTime(condition='s'):   # returns exponentially distributed healing time for a patient who is home

    if condition == 's':                # If stable patient at home, use MU_S
        return random.expovariate(MU_S)
    else:                               # If critical patient forced to go home, use MU_CB / alpha, which will take longer
        alpha = random.uniform(1.25, 1.75)
        return random.expovariate(MU_CB / alpha)


# Global Structures
state = {}          # State Variables
FEL = []            # Future Event List (FEL) - list of events to be processed
event_history = []  # History of events processed

patient_info = {}   # Dictionary to store patient information
next_patient_id = 0 # Next patient ID to be assigned


def schedule_event(time, etype, patientID=None, extra=None):    # Schedule an event
    FEL.append((time, etype, patientID, extra))


def get_next_event():               # Get the next event from the Future Event List (FEL)
    FEL.sort(key=lambda x: x[0])    # Sort the FEL by event time
    return FEL.pop(0)               # remove and return the first event


def Arrival(event):
    global state, FEL, next_patient_id, patient_info

    state['clock'] = event[0]       # Update simulation clock to this event's time

    pid = next_patient_id           # Assign a new patient ID
    next_patient_id += 1            # Increment the patient ID for the next patient
    
    patient_info[pid] = {'arrival_time': state['clock']}    # Record arrival time

    state['totalArrivals'] += 1     # Increment total patients arrived

    next_arr_time = state['clock'] + GenerateInterarrival() # Schedule the next arrival
    schedule_event(next_arr_time, 'Arrival')                # Add to FEL

    if state['busyNurses'] < S:             # If a nurse is available
        state['busyNurses'] += 1            # Increment busy nurses
        finish_time = state['clock'] + GenerateNurseServiceTime()   # Generate service time
        schedule_event(finish_time, 'DepartureTriage', pid)         # Add to FEL
    else:                                   # If no nurses are available, patient must wait   
        state['numberInTriageQueue'] += 1   # Increment the queue count


def DepartureTriage(event):             
    global state, FEL, patient_info

    current_time = event[0]         
    state['clock'] = current_time   # Update clock to this event's time
    pid = event[2]                  # Get the patient ID

    if state['numberInTriageQueue'] > 0:    # If there are patients waiting in triage queue
        state['numberInTriageQueue'] -= 1   # Decrement the queue count
        global next_patient_id              
        new_pid = next_patient_id           # Assign a new patient ID

        next_patient_id += 1                # Increment the patient ID for the next patient
        patient_info[new_pid] = {'arrival_time': current_time}      # Record arrival time

        finish2 = current_time + GenerateNurseServiceTime()         # Generate service time
        schedule_event(finish2, 'DepartureTriage', new_pid)         # Add to FEL
    else:                           # If no patients are waiting, decrement busy nurses
        state['busyNurses'] -= 1    

    if random.random() < P_STABLE:  # If the patient is stable, recover at home
        state['stableCount'] += 1   # Increment stable count
        done_home = current_time + GenerateHomeHealingTime('s')     # Generate home healing time
        schedule_event(done_home, 'RecoveryHome', pid, {'isStable': True})  # Add to FEL
    else:               # If the patient is critical
        state['criticalCount'] += 1     # Increment critical count
        if state['occupiedBeds'] < K:   # If a hospital bed is available, admit the patient
            state['occupiedBeds'] += 1          
            done_bed = current_time + GenerateHospitalHealingTime()
            schedule_event(done_bed, 'TreatedAtHospital', pid)
        else:                           # If no bed is available, send the patient home
            state['rejectedCritical'] += 1
            done_home = current_time + GenerateHomeHealingTime('c')
            schedule_event(done_home, 'RecoveryHome', pid, {'isStable': False})


def TreatedAtHospital(event):
    global state, FEL, patient_info
    
    current_time = event[0]     
    state['clock'] = current_time   # Update clock to this event's time
    pid = event[2]                  # Get the patient ID

    state['occupiedBeds'] -= 1      # Decrement the number of occupied beds
    state['patientsHealed'] += 1    # Increment the number of patients healed

    arr_time = patient_info[pid]['arrival_time'] if pid in patient_info else 0.0    # Record arrival time
    state['sum_time_in_system'] += (current_time - arr_time)                        # Update time in system
    state['count_patients_finished'] += 1       # Increment count of patients finished


def RecoveryHome(event):
    global state, FEL, patient_info

    current_time = event[0]
    state['clock'] = current_time   # Update clock to this event's time
    pid = event[2]                  # Get the patient ID

    state['patientsHealed'] += 1    # Increment the number of patients healed

    arr_time = patient_info[pid]['arrival_time'] if pid in patient_info else 0.0    # Record arrival time
    state['sum_time_in_system'] += (current_time - arr_time)                        # Update time in system 
    state['count_patients_finished'] += 1       # Increment count of patients finished


def apply_initial_condition(initial_condition):
    global state, FEL, next_patient_id, patient_info

    if initial_condition == 'half':     # 'half' initial condition
        half_nurses = math.floor(S/2)   # Half the number of nurses
        for i in range(half_nurses):    
            pid = next_patient_id       # Assign a new patient ID
            next_patient_id += 1        # Increment the patient ID for the next patient
            patient_info[pid] = {'arrival_time': 0.0}   # Record arrival time

            finish_t = random.expovariate(MU_T)         # Generate service time
            schedule_event(finish_t, 'DepartureTriage', pid)    # Add to FEL
        state['busyNurses'] = half_nurses               # Half the number of busy nurses

        half_beds = math.floor(K/2)     # Half the number of beds
        for i in range(half_beds):
            pid = next_patient_id       # Assign a new patient ID
            next_patient_id += 1        # Increment the patient ID for the next patient
            patient_info[pid] = {'arrival_time': 0.0}   # Record arrival time

            finish_b = random.expovariate(MU_CB)        # Generate service time
            schedule_event(finish_b, 'TreatedAtHospital', pid)  # Add to FEL
        state['occupiedBeds'] = half_beds               # Half the number of occupied beds

    elif initial_condition == 'full':   # 'full' initial condition
        for i in range(S):              # Same process as above, but for all nurses
            pid = next_patient_id
            next_patient_id += 1
            patient_info[pid] = {'arrival_time': 0.0}

            finish_t = random.expovariate(MU_T)
            schedule_event(finish_t, 'DepartureTriage', pid)
        state['busyNurses'] = S

        for i in range(K):              # Same process as above, but for all beds
            pid = next_patient_id
            next_patient_id += 1
            patient_info[pid] = {'arrival_time': 0.0}

            finish_b = random.expovariate(MU_CB)
            schedule_event(finish_b, 'TreatedAtHospital', pid)
        state['occupiedBeds'] = K
    else:                               # 'empty' initial condition
        pass

# Main simulation function
def run_simulation(target_healed, max_events, initial_condition):
    global state, FEL, event_history, patient_info, next_patient_id

    state = {                   # State Variables
        'clock': 0.0,
        'numberInTriageQueue': 0,
        'busyNurses': 0,
        'occupiedBeds': 0,
        'patientsHealed': 0,
        'totalArrivals': 0,
        'rejectedCritical': 0,
        'stableCount': 0,
        'criticalCount': 0,
        'sum_time_in_system': 0.0,
        'count_patients_finished': 0,
    }
    FEL = []                # Future Event List (FEL) - list of events to be processed
    event_history = []      # History of events processed
    patient_info = {}       # Dictionary to store patient information
    next_patient_id = 0     # Next patient ID to be assigned

    # Time-based accumulators
    last_event_time = 0.0          
    total_time_triage_empty = 0.0
    total_time_beds_empty = 0.0
    total_time_both_empty = 0.0
    nurse_busy_time = 0.0
    beds_occupied_time = 0.0

    first_arrival = GenerateInterarrival()      # 1) Schedule the first arrival
    schedule_event(first_arrival, 'Arrival')

    apply_initial_condition(initial_condition)  # 2) Apply initial condition

    # 3) Main loop
    num_events = 0
    while state['patientsHealed'] < target_healed and num_events < max_events and len(FEL) > 0:
        e = get_next_event()        # Get the next event from the Future Event List (FEL)
        current_time = e[0]         # Update simulation clock to this event's time
        event_type = e[1]           # Get the event type

        delta = current_time - last_event_time      # Calculate time since last event

        if state['busyNurses']==0 and state['numberInTriageQueue']==0:  # If triage is empty
            total_time_triage_empty += delta

        if state['occupiedBeds']==0:                                    # If beds are empty
            total_time_beds_empty += delta

        if (state['busyNurses']==0 and state['numberInTriageQueue']==0  # If both triage and beds are empty
            and state['occupiedBeds']==0):
            total_time_both_empty += delta

        nurse_busy_time += (state['busyNurses'] * delta)        # Update nurse busy time
        beds_occupied_time += (state['occupiedBeds'] * delta)   # Update bed occupied time   

        last_event_time = current_time          # Update last event time

        if event_type == 'Arrival':             # Process arrival event
            Arrival(e)
        elif event_type == 'DepartureTriage':   # Process departure from triage event
            DepartureTriage(e)
        elif event_type == 'TreatedAtHospital': # Process treated at hospital event
            TreatedAtHospital(e)
        elif event_type == 'RecoveryHome':      # Process recovery at home event
            RecoveryHome(e)
        else:
            pass

        num_events += 1     # Increment event count
        
        if num_events <= 20:    # Store event history for the first 20 events    
            event_history.append({
                'Event#': num_events,
                'Clock': round(state['clock'], 4),
                'EventType': event_type,
                'TriageQueue': state['numberInTriageQueue'],
                'BusyNurses': state['busyNurses'],
                'OccupiedBeds': state['occupiedBeds'],
                'PatientsHealed': state['patientsHealed']
            })

    # End loop
    sim_time = state['clock']   

    # 1) Probability of triage being empty
    prob_triage_empty = total_time_triage_empty/sim_time if sim_time>0 else 0.0

    # 2) Probability of beds being empty
    prob_beds_empty = total_time_beds_empty/sim_time if sim_time>0 else 0.0

    # 3) Probability of both being empty
    prob_both_empty = total_time_both_empty/sim_time if sim_time>0 else 0.0

    # 4) Average nurse utilization
    avg_nurse_util = (nurse_busy_time/(sim_time * S)) if sim_time>0 else 0.0

    # 5) Average occupied beds
    avg_occupied_beds = (beds_occupied_time/sim_time) if sim_time>0 else 0.0

    # 6) Proportion critical rejected
    if state['criticalCount']>0:
        crit_reject_rate = state['rejectedCritical']/state['criticalCount']
    else:
        crit_reject_rate = 0.0

    # 7) Proportion treated at home
    forced_home = state['rejectedCritical']
    stable_home = state['stableCount']
    total_home = forced_home + stable_home
    if state['totalArrivals']>0:
        prop_home = total_home / state['totalArrivals']
    else:
        prop_home = 0.0

    # 8) Average time in system
    if state['count_patients_finished']>0:
        avg_time_sys = state['sum_time_in_system']/state['count_patients_finished']
    else:
        avg_time_sys = 0.0

    results = {
        'final_clock': sim_time,
        'healed': state['patientsHealed'],
        'arrived': state['totalArrivals'],

        'prob_triage_empty': prob_triage_empty,
        'prob_beds_empty': prob_beds_empty,
        'prob_both_empty': prob_both_empty,

        'avg_nurse_util': avg_nurse_util,
        'avg_occupied_beds': avg_occupied_beds,

        'critical_arrivals': state['criticalCount'],
        'rejected_critical': state['rejectedCritical'],
        'crit_reject_rate': crit_reject_rate,

        'stable_count': state['stableCount'],
        'prop_treated_home': prop_home,

        'avg_time_in_system': avg_time_sys,
    }

    return event_history, results


def confidence_interval_generation():
    num_runs = 20
    all_results = []
    base_seed = 4040800189

    for i in range(num_runs):
        SEED = base_seed + i  # update seed for each run
        random.seed(SEED)
        # Run the simulation with target healed patients = 1000 and 'full' initial condition
        _, stats = run_simulation(target_healed=200, max_events=9999999, initial_condition='full')
        all_results.append(stats)

    # Collect metrics of interest across runs
    metrics = {
        "Probability triage empty": [r['prob_triage_empty'] for r in all_results],
        "Probability beds empty": [r['prob_beds_empty'] for r in all_results],
        "Probability both empty": [r['prob_both_empty'] for r in all_results],
        "Proportion of critical patients rejected": [r['crit_reject_rate'] for r in all_results],
        "Average nurse utilization": [r['avg_nurse_util'] for r in all_results],
        "Average number of occupied beds": [r['avg_occupied_beds'] for r in all_results],
        "Proportion of patients treated at home": [r['prop_treated_home'] for r in all_results],
        "Average time a sick person gets better": [r['avg_time_in_system'] for r in all_results],
    }

    # Compute mean and 95% confidence intervals (using t-distribution, t ≈ 2.093 for df=19)
    t_value = 2.093
    summary = {}
    for metric, values in metrics.items():
        n = len(values)
        mean_val = sum(values) / n
        variance = sum((x - mean_val) ** 2 for x in values) / (n - 1)
        std_error = math.sqrt(variance) / math.sqrt(n)
        ci_lower = mean_val - t_value * std_error
        ci_upper = mean_val + t_value * std_error
        summary[metric] = (mean_val, ci_lower, ci_upper)

    # Print the summary table
    print("| Performance Metric                          | Mean    | 95% CI (Lower) | 95% CI (Upper) |")
    print("|--------------------------------------------|---------|---------------|---------------|")
    for metric, (mean_val, ci_lower, ci_upper) in summary.items():
        print(f"| {metric:<42} | {mean_val:7.4f} | {ci_lower:13.4f} | {ci_upper:13.4f} |")


if __name__ == "__main__":
    # Target number of healed patients can be set here (20, 200, 1000, etc.)
    # Initial condition can be set here ('empty', 'half', or 'full')
    ev_hist, stats = run_simulation(target_healed=1000, max_events=9999999, initial_condition='full')

    print("First 20 events (or fewer):")
    for row in ev_hist:
        print(row)

    print("\nFinal Stats:")
    for k,v in stats.items():
        print(f"{k}: {v}")

    # To see the confidence intervals, uncomment the following line:
    # confidence_interval_generation()



First 20 events (or fewer):
{'Event#': 1, 'Clock': 0.6024, 'EventType': 'DepartureTriage', 'TriageQueue': 0, 'BusyNurses': 2, 'OccupiedBeds': 9, 'PatientsHealed': 0}
{'Event#': 2, 'Clock': 0.7858, 'EventType': 'TreatedAtHospital', 'TriageQueue': 0, 'BusyNurses': 2, 'OccupiedBeds': 8, 'PatientsHealed': 1}
{'Event#': 3, 'Clock': 1.3889, 'EventType': 'DepartureTriage', 'TriageQueue': 0, 'BusyNurses': 1, 'OccupiedBeds': 9, 'PatientsHealed': 1}
{'Event#': 4, 'Clock': 2.9758, 'EventType': 'Arrival', 'TriageQueue': 0, 'BusyNurses': 2, 'OccupiedBeds': 9, 'PatientsHealed': 1}
{'Event#': 5, 'Clock': 3.5145, 'EventType': 'DepartureTriage', 'TriageQueue': 0, 'BusyNurses': 1, 'OccupiedBeds': 9, 'PatientsHealed': 1}
{'Event#': 6, 'Clock': 3.6621, 'EventType': 'TreatedAtHospital', 'TriageQueue': 0, 'BusyNurses': 1, 'OccupiedBeds': 8, 'PatientsHealed': 2}
{'Event#': 7, 'Clock': 4.2505, 'EventType': 'TreatedAtHospital', 'TriageQueue': 0, 'BusyNurses': 1, 'OccupiedBeds': 7, 'PatientsHealed': 3}
{'Event#